# Descriptive analysis of MR3 mobility projects

(with a bit of exploratory analysis)

In [3]:
import pandas as pd
import numpy as np
import pandas_profiling

import pickle

import matplotlib.pyplot as plt

import os
import glob

In [4]:
# hyperparamteres

num_top_opt = 8 # the selected optimal number of topics based on LDA analysis

### MR3 mobility projects & their LDA features

In [15]:
df_mobility_projects = pd.read_csv("mr3_results/final_project_selection_with_lda_features_numtopics"\
                                   + str(num_top_opt) + ".csv")
df_mobility_projects.drop("Unnamed: 0", axis=1, inplace=True)

df_mobility_projects.head(3)

,id,acronym,title,objective,summary,workPerformed,finalResults,text,clean_text,framework,t0,t1,t2,t3,t4,t5,t6,t7
0,120009,SE2A,"Nanoelectronics for Safe, Fuel Efficient and E...",The societal need for a transport infrastructu...,NaN,NaN,NaN,"Nanoelectronics for Safe, Fuel Efficient and E...",nanoelectronics safe fuel efficient environmen...,FP7,0.189020,0.000450,0.001387,0.068556,0.270317,0.174095,0.293161,0.003014
1,204513,DCBIF,Flight dynamics and control of birds and insects,"Insects bristle with sensors, but how do they ...",This project aimed to develop an understanding...,NaN,NaN,Flight dynamics and control of birds and insec...,flight dynamic control bird insect insects bri...,FP7,0.219498,0.000130,0.114235,0.108499,0.435106,0.120905,0.000812,0.000814
2,205768,AGAPE,ACARE Goals Progress Evaluation,ACARE (the Advisory Council for Aeronautics Re...,NaN,NaN,NaN,ACARE Goals Progress Evaluation ACARE (the Adv...,acare goals progress evaluation acare advisory...,FP7,0.001863,0.000365,0.001124,0.000800,0.052357,0.306412,0.634706,0.002373


In [16]:
df_mobility_projects.shape

(926, 18)

In [17]:
df_mobility_projects.describe()

,id,t0,t1,t2,t3,t4,t5,t6,t7
count,926.000000,926.000000,926.000000,926.000000,926.000000,926.000000,926.000000,926.000000,926.000000
mean,554398.146868,0.102137,0.030913,0.119484,0.056603,0.103021,0.260578,0.160397,0.166868
std,235010.671220,0.132370,0.083482,0.192735,0.103611,0.189124,0.198878,0.187693,0.197083
min,120009.000000,0.000032,0.000006,0.000022,0.000014,0.000019,0.000352,0.000084,0.000040
25%,285119.500000,0.002057,0.000096,0.000772,0.000243,0.000406,0.096867,0.018083,0.003544
50%,652361.500000,0.038630,0.000326,0.014377,0.001639,0.006584,0.225954,0.099937,0.074994
75%,763667.750000,0.171800,0.006833,0.155771,0.062384,0.106611,0.391520,0.221008,0.286965
max,878052.000000,0.876684,0.603449,0.851547,0.697143,0.908642,0.896735,0.907718,0.874336


### Read full H2020 and FP7 dataframes

In [18]:
h2020 = pd.read_csv("datasets/data_may20/h2020_clean/h2020_full.csv")
fp7 = pd.read_csv("datasets/data_may20/fp7_clean/fp7_full.csv")

In [19]:
h2020.head(2)

,rcn,id,acronym,status,programme,topics,frameworkProgramme,title,startDate,endDate,...,rcn_report,title_report,summary,workPerformed,finalResults,lastUpdateDate,projectID,projectAcronym,relatedFile,url
0,229267,894593,ICARUS,SIGNED,H2020-EU.3.4.7.,SESAR-ER4-31-2019,H2020,INTEGRATED COMMON ALTITUDE REFERENCE SYSTEM FO...,2020-05-01,2022-07-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,229284,897004,ISLand,SIGNED,H2020-EU.1.3.2.,MSCA-IF-2019,H2020,Isolation and Segregation Landscape. Archaeolo...,2020-11-01,2023-10-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
print(h2020.columns.values)

['rcn' 'id' 'acronym' 'status' 'programme' 'topics' 'frameworkProgramme'
 'title' 'startDate' 'endDate' 'projectUrl' 'objective' 'totalCost'
 'ecMaxContribution' 'call' 'fundingScheme' 'coordinator'
 'coordinatorCountry' 'participants' 'participantCountries' 'subjects'
 'rcn_report' 'title_report' 'summary' 'workPerformed' 'finalResults'
 'lastUpdateDate' 'projectID' 'projectAcronym' 'relatedFile' 'url']


In [22]:
print(fp7.columns.values)

['rcn' 'id' 'acronym' 'status' 'programme' 'topics' 'frameworkProgramme'
 'title' 'startDate' 'endDate' 'projectUrl' 'objective' 'totalCost'
 'ecMaxContribution' 'call' 'fundingScheme' 'coordinator'
 'coordinatorCountry' 'participants' 'participantCountries' 'subjects'
 'summary' 'lastUpdateDate' 'rcn_report' 'title_report' 'projectAcronym']


### Other data

#### FP7
- orgs = cordis-fp7organizations
- projects = cordis-fp7projects
- reports = cordis-fp7reports

#### H2020

### FP7

In [24]:
fp7_mobility = df_mobility_projects[df_mobility_projects.framework == "FP7"]
fp7_mobility.shape

(407, 18)

In [32]:
path = "datasets/data_may20/fp7_raw/"
csvs = sorted([f for f in glob.glob(path + "*.csv", recursive = False)])
print(len(csvs))
csvs

3


['datasets/data_may20/fp7_raw/cordis-fp7organizations.csv',
 'datasets/data_may20/fp7_raw/cordis-fp7projects.csv',
 'datasets/data_may20/fp7_raw/cordis-fp7reports.csv']

In [33]:
orgs_path, projs_path, reports_path = csvs

In [37]:
# reads_csv


### H2020

In [25]:
h2020_mobility = df_mobility_projects[df_mobility_projects.framework == "H2020"]
h2020_mobility.shape

(519, 18)